# Python处理PDF详解 - 自动化与数据提取优势

## 📚 学习目标
本教程重点讲解Python处理PDF的**独有优势**，而非简单的PDF编辑功能。

### Python处理PDF的核心优势：
1. **批量自动化处理** - 一次性处理成百上千个PDF文件
2. **数据提取与分析** - 从PDF中提取结构化数据并进行分析
3. **程序化生成报告** - 根据数据动态生成PDF报告
4. **与其他工具集成** - 结合pandas、numpy等进行数据处理
5. **定制化处理逻辑** - 实现复杂的条件判断和数据处理

### 主要学习的库：
- **PyPDF2** - PDF基础操作（合并、拆分、旋转）
- **pdfplumber** - 精确提取文本和表格数据
- **PyMuPDF (fitz)** - 高性能PDF处理
- **reportlab** - 程序化生成PDF文档


## 📦 安装必要的库

首先安装我们需要的PDF处理库：


In [ ]:
# 安装PDF处理相关的库
import sys
import subprocess

# 需要安装的库列表
libraries = [
    'PyPDF2',      # PDF基础操作
    'pdfplumber',  # 精确提取文本和表格
    'PyMuPDF',     # 高性能PDF处理（导入时使用fitz）
    'reportlab',   # 生成PDF
    'pandas',      # 数据处理（配合使用）
]

print("正在检查并安装必要的库...\\n")
for lib in libraries:
    try:
        __import__(lib.lower().replace('-', '_'))
        print(f"✓ {lib} 已安装")
    except ImportError:
        print(f"⏳ 正在安装 {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"✓ {lib} 安装完成")

print("\\n所有库已准备就绪！")


## 导入所需库


In [ ]:
# 导入PDF处理库
import PyPDF2
import pdfplumber
import fitz  # PyMuPDF的导入名称是fitz
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# 导入辅助库
import pandas as pd
import os
from pathlib import Path
import re

print("✓ 所有库导入成功！")
print(f"PyPDF2 版本: {PyPDF2.__version__}")
print(f"PyMuPDF 版本: {fitz.__version__}")


---
# 第一部分：PyPDF2 - PDF基础操作

## ⭐ Python优势：批量自动化处理

### 应用场景：
- 批量合并大量PDF文件（如合并所有月度报告）
- 自动拆分PDF文件（如将400页的文档按章节拆分）
- 批量添加水印、页码
- 批量提取特定页面


### 1. 创建示例PDF文件（用于演示）


In [ ]:
# 创建示例PDF文件用于后续操作
from reportlab.pdfgen import canvas

# 创建测试目录
test_dir = Path("pdf_test_files")
test_dir.mkdir(exist_ok=True)

# 创建3个示例PDF文件
for i in range(1, 4):
    c = canvas.Canvas(str(test_dir / f"sample_{i}.pdf"), pagesize=letter)
    c.drawString(100, 750, f"这是示例PDF文件 #{i}")
    c.drawString(100, 730, f"第1页内容")
    c.showPage()
    c.drawString(100, 750, f"这是示例PDF文件 #{i}")
    c.drawString(100, 730, f"第2页内容")
    c.showPage()
    c.save()

print("✓ 已创建3个示例PDF文件，每个文件2页")
print(f"文件位置: {test_dir.absolute()}")


### ⭐ 重点复习：批量合并PDF文件

**Python的独特优势：**
- 一次性合并成百上千个PDF文件
- 可以根据文件名排序、筛选特定文件
- 可以添加逻辑判断（如只合并特定日期的文件）
- 完全自动化，无需手动操作

**实际应用场景：**
- 合并所有季度财务报告
- 合并客户订单PDF生成汇总文件
- 合并扫描文档形成完整档案


In [ ]:
# 批量合并多个PDF文件
def merge_pdfs(pdf_files, output_filename):
    """
    批量合并PDF文件
    
    参数：
        pdf_files: PDF文件路径列表
        output_filename: 输出文件名
    """
    # 创建PDF合并器对象
    merger = PyPDF2.PdfMerger()
    
    # 遍历所有PDF文件并添加到合并器
    for pdf_file in pdf_files:
        print(f"正在添加: {pdf_file}")
        merger.append(str(pdf_file))
    
    # 写入合并后的PDF
    merger.write(output_filename)
    merger.close()
    
    print(f"\\n✓ 成功合并 {len(pdf_files)} 个PDF文件")
    print(f"输出文件: {output_filename}")

# 示例：合并所有sample开头的PDF文件
pdf_files = sorted(test_dir.glob("sample_*.pdf"))  # 自动排序
merge_pdfs(pdf_files, test_dir / "merged_all.pdf")

# 验证合并结果
with open(test_dir / "merged_all.pdf", 'rb') as f:
    reader = PyPDF2.PdfReader(f)
    print(f"\\n合并后的PDF共有 {len(reader.pages)} 页")


### ⭐ 重点复习：智能拆分PDF文件

**Python的独特优势：**
- 可以根据内容自动判断拆分点
- 批量拆分多个大型PDF文件
- 灵活控制拆分逻辑（按页数、按章节、按关键词）

**实际应用场景：**
- 将大型合同文档按条款拆分
- 将扫描的多份文档拆分成单独文件
- 按章节自动拆分教材


In [ ]:
# 智能拆分PDF文件
def split_pdf(input_pdf, pages_per_file=2, output_prefix="split"):
    """
    按指定页数拆分PDF文件
    
    参数：
        input_pdf: 输入PDF文件路径
        pages_per_file: 每个文件包含的页数
        output_prefix: 输出文件前缀
    """
    with open(input_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        total_pages = len(reader.pages)
        
        print(f"原文件共 {total_pages} 页")
        print(f"将按每 {pages_per_file} 页拆分\\n")
        
        # 计算需要拆分成多少个文件
        num_files = (total_pages + pages_per_file - 1) // pages_per_file
        
        for file_num in range(num_files):
            # 创建新的PDF写入器
            writer = PyPDF2.PdfWriter()
            
            # 计算起始和结束页
            start_page = file_num * pages_per_file
            end_page = min(start_page + pages_per_file, total_pages)
            
            # 添加页面
            for page_num in range(start_page, end_page):
                writer.add_page(reader.pages[page_num])
            
            # 保存拆分后的文件
            output_file = test_dir / f"{output_prefix}_{file_num + 1}.pdf"
            with open(output_file, 'wb') as output:
                writer.write(output)
            
            print(f"✓ 创建: {output_file.name} (第{start_page + 1}-{end_page}页)")

# 示例：拆分合并后的PDF
split_pdf(test_dir / "merged_all.pdf", pages_per_file=2, output_prefix="chapter")


---
# 第二部分：pdfplumber - 精确数据提取

## ⭐ Python独特优势：结构化数据提取与分析

### pdfplumber的核心优势：
- **精确提取表格数据** - 自动识别表格结构并转换为DataFrame
- **保留文本位置信息** - 知道每个文字的精确坐标
- **提取图表位置** - 识别图表区域
- **批量数据提取** - 从成百上千份PDF中提取结构化数据

### 应用场景：
- 从发票PDF中批量提取金额、日期等信息
- 从报表PDF中提取表格数据进行分析
- 从合同中提取关键条款
- 从简历中提取结构化信息


### 1. 创建包含表格的示例PDF


In [ ]:
# 创建包含表格的示例PDF
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# 创建示例销售数据PDF
pdf_file = test_dir / "sales_report.pdf"
doc = SimpleDocTemplate(str(pdf_file), pagesize=letter)

# 示例销售数据
data = [
    ['产品', '数量', '单价', '总额'],
    ['产品A', '100', '50', '5000'],
    ['产品B', '150', '80', '12000'],
    ['产品C', '200', '30', '6000'],
    ['总计', '450', '-', '23000'],
]

# 创建表格
table = Table(data)
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 14),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
]))

doc.build([table])
print(f"✓ 已创建包含表格的销售报告PDF")
print(f"文件位置: {pdf_file}")


### ⭐ 重点复习：精确提取表格数据

**pdfplumber的独特优势：**
- 自动识别表格结构（无需手动标注）
- 直接转换为Python数据结构（列表、DataFrame）
- 可以处理复杂的合并单元格
- 批量提取多个PDF中的表格

**与手动操作对比：**
- 手动：打开PDF → 复制表格 → 粘贴到Excel → 清理格式（每个文件5-10分钟）
- Python：一行代码，批量处理1000个文件，只需几分钟

**实际应用场景：**
- 从所有月度报表中提取销售数据
- 从发票PDF中批量提取明细
- 从财务报告中提取财务指标


In [ ]:
# 使用pdfplumber提取表格数据
def extract_tables_from_pdf(pdf_path):
    """
    从PDF中提取所有表格数据
    
    返回：表格数据列表
    """
    tables_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        print(f"正在处理: {pdf_path.name}")
        print(f"总页数: {len(pdf.pages)}\\n")
        
        # 遍历每一页
        for page_num, page in enumerate(pdf.pages, 1):
            # 提取表格
            tables = page.extract_tables()
            
            if tables:
                print(f"第{page_num}页找到 {len(tables)} 个表格")
                
                for table_num, table in enumerate(tables, 1):
                    print(f"\\n表格 {table_num}:")
                    
                    # 转换为DataFrame便于处理
                    df = pd.DataFrame(table[1:], columns=table[0])
                    print(df.to_string(index=False))
                    
                    tables_data.append({
                        'page': page_num,
                        'table_num': table_num,
                        'data': df
                    })
    
    return tables_data

# 提取表格数据
tables = extract_tables_from_pdf(test_dir / "sales_report.pdf")

# 演示数据分析
if tables:
    df = tables[0]['data']
    print("\\n\\n📊 数据分析示例：")
    print(f"产品种类: {len(df) - 1}")
    
    # 注意：这里的数值是字符串，需要转换
    try:
        df['数量'] = pd.to_numeric(df['数量'])
        df['总额'] = pd.to_numeric(df['总额'])
        print(f"平均销售额: {df['总额'].iloc[:-1].mean():.2f}")
        print(f"最高销售额产品: {df.loc[df['总额'].iloc[:-1].idxmax(), '产品']}")
    except:
        print("（数据格式需要进一步清理）")


---
# 第三部分：PyMuPDF (fitz) - 高性能处理

## ⭐ Python独特优势：图像提取与高级处理

### PyMuPDF的核心优势：
- **速度快** - 处理大型PDF的速度是PyPDF2的数倍
- **提取图片** - 批量提取PDF中的所有图片
- **获取精确位置** - 获取文本、图片的精确坐标
- **渲染为图片** - 将PDF页面转换为图片（用于预览、OCR等）

### 应用场景：
- 批量提取PDF中的图片（如提取产品目录中的所有图片）
- 将PDF转换为图片进行OCR识别
- 高性能处理大型PDF文档
- 提取带格式的文本信息


### ⭐ 重点复习：将PDF页面转换为图片

**Python的独特优势：**
- 将PDF转换为图片用于OCR识别
- 生成PDF预览缩略图
- 用于图像处理和分析

**实际应用场景：**
- 对扫描版PDF进行OCR文字识别
- 生成PDF文档的预览图
- 将PDF内容用于计算机视觉分析


In [ ]:
# 将PDF页面转换为图片
def pdf_to_images(pdf_path, output_dir=None, dpi=150):
    """
    将PDF的每一页转换为图片
    
    参数：
        pdf_path: PDF文件路径
        output_dir: 输出目录
        dpi: 分辨率（越高越清晰，但文件越大）
    """
    if output_dir is None:
        output_dir = Path(pdf_path).parent / "pdf_as_images"
    
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    doc = fitz.open(pdf_path)
    
    print(f"正在转换: {pdf_path}")
    print(f"总页数: {len(doc)}")
    print(f"分辨率: {dpi} DPI\\n")
    
    zoom = dpi / 72  # 标准PDF是72 DPI
    matrix = fitz.Matrix(zoom, zoom)
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        
        # 渲染页面为图片
        pix = page.get_pixmap(matrix=matrix)
        
        # 保存图片
        image_filename = output_dir / f"page_{page_num + 1}.png"
        pix.save(str(image_filename))
        
        print(f"✓ 已转换第 {page_num + 1} 页: {image_filename.name}")
    
    doc.close()
    
    print(f"\\n✓ 转换完成！")
    print(f"保存位置: {output_dir.absolute()}")

# 示例：将PDF转换为图片
pdf_to_images(test_dir / "sales_report.pdf", dpi=150)

print("\\n💡 DPI选择建议：")
print("- 72-96 DPI: 网页预览（文件小，速度快）")
print("- 150 DPI: 一般用途（平衡质量和文件大小）")
print("- 300 DPI: 高质量（用于打印或OCR识别）")


---
# 第四部分：reportlab - 程序化生成PDF

## ⭐ Python独特优势：动态生成PDF报告

### reportlab的核心优势：
- **完全程序化** - 根据数据动态生成PDF
- **模板化报告** - 定义一次模板，生成无数份报告
- **自动化报告** - 定时生成日报、周报、月报
- **个性化内容** - 为每个客户生成定制化报告

### 应用场景：
- 自动生成销售报告、财务报表
- 批量生成个性化证书、发票
- 根据数据库数据生成PDF报告
- 创建数据可视化报告（结合matplotlib）


### ⭐ 重点复习：根据数据动态生成PDF报告

**这是reportlab的核心价值！**

**与手动制作对比：**
- 手动：在Word中制作报告 → 导出为PDF（每份报告15-30分钟）
- Python：定义模板 → 运行脚本生成1000份报告（几分钟）

**实际应用场景：**
- 每天自动生成销售日报
- 为每个客户生成个性化月度报告
- 根据数据库数据批量生成发票


In [ ]:
# 动态生成PDF报告
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.lib import colors
from datetime import datetime

def generate_sales_report(sales_data, output_filename, report_date=None):
    """
    根据销售数据动态生成PDF报告
    
    参数：
        sales_data: 销售数据（列表或DataFrame）
        output_filename: 输出文件名
        report_date: 报告日期
    """
    if report_date is None:
        report_date = datetime.now().strftime("%Y-%m-%d")
    
    # 创建PDF文档
    doc = SimpleDocTemplate(output_filename, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    
    # 添加标题
    title = Paragraph(f"<b>Sales Report - {report_date}</b>", styles['Title'])
    elements.append(title)
    elements.append(Spacer(1, 0.3*inch))
    
    # 添加摘要信息
    if isinstance(sales_data, pd.DataFrame):
        total_sales = sales_data['总额'].sum()
        num_products = len(sales_data)
    else:
        num_products = len(sales_data) - 1
        total_sales = sum([int(row[3]) for row in sales_data[1:-1]])
    
    summary = Paragraph(f"<b>Summary:</b> {num_products} products, Total Sales: ${total_sales:,}", 
                       styles['Normal'])
    elements.append(summary)
    elements.append(Spacer(1, 0.3*inch))
    
    # 创建销售数据表格
    if isinstance(sales_data, pd.DataFrame):
        table_data = [sales_data.columns.tolist()] + sales_data.values.tolist()
    else:
        table_data = sales_data
    
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -2), colors.beige),
        ('BACKGROUND', (0, -1), (-1, -1), colors.lightgrey),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    
    # 添加页脚
    elements.append(Spacer(1, 0.5*inch))
    footer = Paragraph(f"<i>Report generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</i>",
                      styles['Normal'])
    elements.append(footer)
    
    # 构建PDF
    doc.build(elements)
    print(f"✓ 报告已生成: {output_filename}")

# 示例数据
sample_sales = pd.DataFrame({
    '产品': ['Product A', 'Product B', 'Product C'],
    '数量': [120, 85, 200],
    '单价': [50, 80, 30],
    '总额': [6000, 6800, 6000]
})

# 生成报告
generate_sales_report(sample_sales, test_dir / "automated_report.pdf", "2024-01-15")

print("\\n💡 实际应用：")
print("你可以将这个函数集成到定时任务中：")
print("- 每天自动从数据库提取数据并生成报告")
print("- 为每个销售人员生成个性化业绩报告")
print("- 批量生成客户对账单")


---
# 总结：Python处理PDF的独特优势

## 🎯 核心优势总结

### 1. 自动化与批量处理
- ✅ 批量合并/拆分成百上千个PDF
- ✅ 批量提取数据，无需手动操作
- ✅ 定时自动生成报告
- ❌ PDF软件：只能一个一个手动处理

### 2. 数据提取与分析
- ✅ 精确提取表格数据并转换为DataFrame
- ✅ 使用正则表达式提取特定信息
- ✅ 结合pandas进行数据分析
- ❌ PDF软件：只能复制粘贴，无法直接分析

### 3. 程序化生成PDF
- ✅ 根据数据动态生成PDF
- ✅ 批量生成个性化文档
- ✅ 模板化报告系统
- ❌ PDF软件：每个文档都需要手动制作

### 4. 集成与扩展
- ✅ 与数据库集成
- ✅ 与Web应用集成
- ✅ 与数据分析工具集成
- ❌ PDF软件：功能孤立，难以集成

## 📚 库的选择建议

| 需求 | 推荐库 | 理由 |
|-----|--------|------|
| 基础操作（合并/拆分） | PyPDF2 | 简单易用 |
| 数据提取（文本/表格） | pdfplumber | 提取精确 |
| 高性能处理/图片提取 | PyMuPDF | 速度快 |
| 生成PDF | reportlab | 功能强大 |

## 🚀 实际应用建议

1. **从小做起** - 先自动化一个简单任务
2. **模板化** - 将常用操作封装成函数
3. **批量处理** - 充分利用Python的批量处理能力
4. **错误处理** - 添加异常处理，确保稳定性
5. **日志记录** - 记录处理过程，便于调试

## 💡 下一步学习方向

- 结合OCR技术处理扫描版PDF
- 使用NLP技术进行文本分析
- 创建Web应用提供PDF处理服务
- 结合matplotlib创建数据可视化报告

---

**记住：Python处理PDF的核心价值不是替代PDF编辑软件，而是实现自动化、批量化、智能化的PDF数据处理！**


## 清理测试文件

如果需要清理测试文件，可以运行下面的代码：


In [ ]:
# 如果需要清理测试文件，取消下面代码的注释
"""
import shutil

if test_dir.exists():
    shutil.rmtree(test_dir)
    print(f"✓ 已清理测试文件夹: {test_dir}")
"""
